# LLM API 실습


In [2]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

## 1. 텍스트 생성
### 단일입력 사용

In [3]:
from google import genai

# import google.generativeai as genai

# 클라이언트 인스턴스 생성
client = genai.Client(api_key=GOOGLE_API_KEY)

# singturn 한번의 질문으로 쿼리를 생성
# multiturn 여러번

# 모델을 지정하고 컨텐츠의 답변을 요청
response = client.models.generate_content(
    # vertexai=False,            # vertex ai에서 활용할때 옵션 추가
    model="gemini-2.5-flash",    # model version
    contents="남아프리카공화국의 수도는 어디인가요?",   # 질문(프롬프트)
)
# response변수에 답변이 담겨 있음
print(response.text)

남아프리카 공화국은 특이하게도 **세 개의 수도**를 가지고 있습니다. 각 수도는 다른 기능을 담당합니다.

1.  **프리토리아 (Pretoria)**: 행정 수도 (Administrative Capital) - 대통령과 정부 부처가 위치합니다.
2.  **케이프타운 (Cape Town)**: 입법 수도 (Legislative Capital) - 의회(국회)가 위치합니다.
3.  **블룸폰테인 (Bloemfontein)**: 사법 수도 (Judicial Capital) - 대법원이 위치합니다.

이 중 일반적으로 '수도'라고 하면 **프리토리아**를 지칭하는 경우가 많습니다.


In [4]:
# 토큰 사용량 정보 출력

# candidates_token_count =10 질문에 대한 토큰
# prompt_token_count=10 답변에 대한 토큰
# thoughts_token_count 내부적으로 쓴 토큰 카운트
# candidates_token_count + prompt_token_count + thoughts_token_count 세 값의 총함은 총사용한 토큰
# 모델에 따라 비용이 다름 현재를 기준으로(홈페이지에서 확인)Input $0.30, Output $2.50인데 입력과 출력의 토큰 가격이 다르다

# query 토큰 수
print(response.usage_metadata.prompt_token_count)

# throughts 토큰 수(생각하는 토큰 수)
print(response.usage_metadata.thoughts_token_count)

# query 토근수 + throughts 토큰 수 + 응답 토근 수
print(response.usage_metadata.total_token_count)

10
203
223


### 시스템 안내 및 기타 구성

In [6]:
from google import genai
from google.genai import types

# 인스턴스 생성
client = genai.Client(api_key=GOOGLE_API_KEY)

# Google Gemini 모델과 직접 상호작용하여 콘텐츠를 생성하는 데 사용되는 핵심 함수
response = client.models.generate_content(
    model="gemini-2.5-flash",
    config=types.GenerateContentConfig(
        # 시스템 인스트럭션 (전체 대화에 걸쳐 LLM의 답변을 제어하는 특별한 프롬프트)
        system_instruction="당신은 선생님이고, 이름은 김민철입니다." # 페르소나(역할)을 지정
    ),
    contents="안녕",
)

print(response.text)

안녕! 👋 무슨 일로 왔니?


### GenerateContentConfig : temperature 설정

In [5]:
from google import genai
from google.genai import types

client = genai.Client(api_key=GOOGLE_API_KEY)

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=["AI가 어떻게 동작하는지 10문장으로 정리해줘"],
    #contents=["Explain how AI works"],
    # temperature를 설정
    # 1에 가까울 수록 무작위성이 높아짐. 0에 가까울 수록 일관적
    config=types.GenerateContentConfig(temperature=0.1),
)
print(response.text)

AI는 컴퓨터가 인간처럼 학습하고 추론하며 문제를 해결하도록 돕는 기술입니다.
이를 위해 AI는 방대한 양의 데이터를 입력받습니다.
이 데이터 속에서 특정 패턴, 규칙, 특징을 스스로 찾아내고 학습합니다.
이 학습 과정은 '알고리즘'이라는 일련의 계산 규칙을 통해 이루어지며, 학습된 결과물이 '모델'입니다.
특히 '인공신경망'은 인간 뇌의 구조를 모방하여 복잡한 패턴을 인식하는 데 주로 사용됩니다.
학습이 완료된 모델은 새로운 데이터가 주어졌을 때, 학습된 패턴을 기반으로 분석합니다.
그리고 예측, 분류, 추천 등 특정 작업을 수행하거나 결정을 내립니다.
AI는 자신의 예측이나 결정이 틀렸을 경우, 피드백을 통해 성능을 개선하며 점차 정확도를 높여갑니다.
결국 AI는 데이터를 통해 지식을 습득하고, 이를 활용하여 주어진 문제를 해결하는 시스템입니다.
이 과정을 통해 인간의 개입 없이도 복잡한 작업을 자동화하고 효율성을 증대시킵니다.


### 멀티모달 입력

In [7]:
from google.genai import types

# 이미지 파일을 읽어와서 바이너리 데이터로 변환
# rb => read binery
with open("marimba.jpg", "rb") as f: # with 파일을 다 읽을 때까지 실행
    image_bytes = f.read()


response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=[
        # 바이너리 타입의 이미지
        types.Part.from_bytes(
            # 바이너리로 읽은 데이터를 첨부하기
            data=image_bytes,
            mime_type="image/jpeg",
        ),
        # text 타입의 프롬프트
        "이 악기에 대해 설명해줘",
    ],
)

print("=== 응답 ===")
print(response.text)
print("\n=== 토큰 사용량 ===")
print(f"입력 토큰: {response.usage_metadata.prompt_token_count}")
print(f"출력 토큰: {response.usage_metadata.candidates_token_count}")
print(f"생각 토큰: {response.usage_metadata.thoughts_token_count}")
print(f"총 토큰: {response.usage_metadata.total_token_count}")

=== 응답 ===
이 악기는 **마림바(Marimba)**입니다.

마림바는 크고 음정이 있는 타악기(pitched percussion instrument)의 일종으로, 주요 특징은 다음과 같습니다.

*   **건반 (Bars):** 두 줄로 배열된 나무 건반(bars)으로 구성되어 있습니다. 이 건반들은 피아노 건반처럼 음높이에 따라 길이가 다르며, 음계에 맞춰 배열되어 있습니다. 마림바의 건반은 주로 로즈우드(rosewood)나 합성 소재로 만들어져 따뜻하고 풍부한 음색을 냅니다.
*   **공명관 (Resonators):** 각 나무 건반 아래에는 길이가 다른 금속 공명관(resonators)이 부착되어 있습니다. 이 튜브들은 건반에서 나는 소리를 증폭시키고, 길게 울리도록(잔향) 도와주는 역할을 합니다.
*   **프레임 (Frame):** 건반과 공명관은 튼튼한 검은색 금속 프레임 위에 놓여 있습니다. 이 프레임은 안정적으로 악기를 지지하며, 사진에서 보이는 것처럼 이동을 위한 바퀴(casters)가 달려 있어 운반이 용이합니다.
*   **브랜드:** 왼쪽 프레임에는 'MUSSER'라는 브랜드 이름이 보입니다. Musser는 마림바, 비브라폰 등 말렛(mallet) 타악기 분야에서 유명한 제조업체 중 하나입니다.
*   **연주 방식:** 솜이나 고무 재질의 말렛(mallets)으로 나무 건반을 두드려 소리를 냅니다. 말렛의 종류에 따라 소리의 질감과 음색이 다양하게 변할 수 있습니다.
*   **음색 및 용도:** 따뜻하고 부드러우며 울림이 좋은 음색을 가지고 있어, 특히 서정적인 선율을 연주하는 데 적합합니다. 클래식 음악, 재즈, 오케스트라, 타악기 앙상블 등 다양한 장르에서 사용됩니다.

요약하자면, 사진 속 악기는 나무 건반과 공명관을 갖춘 **마림바**이며, 풍부하고 따뜻한 소리를 내는 음정 타악기입니다.

=== 토큰 사용량 ===
입력 토큰: 266
출력 토큰: 489
생각 토큰: 1391
총 토큰: 2146


In [8]:
from PIL import Image
from google import genai
from google.genai import types
import io

client = genai.Client(api_key=GOOGLE_API_KEY)

image = Image.open("marimba.jpg")

# Convert image to bytes
# 이미지를 바이너리 코드로 변경
img_byte_arr = io.BytesIO()
image.save(img_byte_arr, format="JPEG")
img_byte_arr = img_byte_arr.getvalue()

response = client.models.generate_content(
    model="gemini-2.5-flash", # flash 모델 사용
    contents=[
        types.Content(
            role="user", # 사용자 입력
            parts=[
                types.Part(text="이 악기에 대해 설명해줘"),
                types.Part(
                    # 대용량 데이터를 jpg를 array로 만들어서 보낸다
                    inline_data=types.Blob(mime_type="image/jpeg", data=img_byte_arr)
                ),
            ],
        )
    ],
)
print(response.text)

이 악기는 **마림바(Marimba)**입니다.

**주요 특징:**

1.  **분류:** 건반 타악기(Keyboard Percussion Instrument)의 일종으로, 나무 건반을 말렛으로 쳐서 소리 내는 악기입니다. 크실로폰(실로폰)과 비슷하게 생겼지만, 음색과 음역대에서 차이가 있습니다.
2.  **외형:**
    *   **나무 건반(음판):** 피아노 건반처럼 두 줄로 배열된 여러 개의 나무 막대(음판)로 구성되어 있습니다. 각 음높이에 따라 길이가 다릅니다. (길이가 길수록 저음, 짧을수록 고음)
    *   **공명관(공명 튜브):** 각 나무 건반 아래에는 소리를 증폭하고 풍부한 울림을 만들어내는 금속 또는 합성수지 재질의 공명관이 달려 있습니다. 저음으로 갈수록 공명관의 길이가 길어집니다.
    *   **프레임:** 튼튼한 금속 프레임 위에 설치되어 있으며, 이동의 편의를 위해 바퀴가 달려 있는 경우가 많습니다.
3.  **연주 방법:** 털실, 고무 등으로 만들어진 **말렛(Mallets)**을 사용하여 나무 건반을 두드려 소리를 냅니다. 연주자는 보통 양손에 2개 또는 4개의 말렛을 쥐고 연주하여 화음이나 복잡한 멜로디를 표현할 수 있습니다.
4.  **음색:** 따뜻하고 부드러우며 깊이 있고 풍부한 울림을 가진 음색이 특징입니다. 크실로폰이 더 밝고 날카로운 소리를 내는 반면, 마림바는 더 서정적이고 감미로운 소리를 냅니다.
5.  **음역:** 보통 4옥타브에서 5옥타브, 심지어 6옥타브에 이르는 넓은 음역을 가지고 있습니다.
6.  **용도:** 오케스트라, 밴드, 솔로 연주, 실내악, 재즈, 현대 음악 등 다양한 장르에서 폭넓게 사용됩니다.


## 2. Thinking

In [11]:
from google import genai
from google.genai import types


client = genai.Client(api_key= GOOGLE_API_KEY)


response = client.models.generate_content(
    model="gemini-2.5-pro",
    # contents="Provide a list of 3 famous physicists and their key contributions",
    # 사용자 입력
    contents="유명한 AI개발자 세 명과 그들의 주요 업적을 알려주세요.",
    config=types.GenerateContentConfig(
        # 최대 사용할 수있는 범위를 설정할 수 있음
         thinking_config=types.ThinkingConfig(thinking_budget=512)

        # Turn off thinking:
        # pro모델의 경우 thinking buget을 0으로 설정못함
        # thinking_config=types.ThinkingConfig(thinking_budget=0)

        # Turn on dynamic thinking:
        # thinking_budget 자동으로 설정
        # thinking_config=types.ThinkingConfig(thinking_budget=-1)
    ),
)

print(response.text)

## AI 분야의 3대 거장과 주요 업적

AI, 특히 딥러닝 분야의 발전을 이끈 "딥러닝의 아버지" 또는 "AI의 대부들(Godfathers of AI)"로 불리는 세 명의 석학이 가장 상징적입니다. 이들은 2018년, 컴퓨터 과학의 노벨상이라 불리는 **튜링상(Turing Award)**을 공동 수상하며 그 공로를 세계적으로 인정받았습니다.

---

### 1. 제프리 힌튼 (Geoffrey Hinton)

![제프리 힌튼](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3b/Geoffrey_Hinton_at_the_Royal_Society.jpg/440px-Geoffrey_Hinton_at_the_Royal_Society.jpg)

**"딥러닝의 개념적 토대를 마련한 선구자"**

*   **소속:** 토론토 대학교 명예교수, 前 구글 부사장 및 엔지니어링 펠로우
*   **주요 업적:**
    *   **역전파 알고리즘(Backpropagation)의 대중화:** 신경망이 스스로 오류를 수정하며 학습할 수 있게 하는 핵심 알고리즘입니다. 힌튼 교수는 이 알고리즘의 중요성을 재발견하고 널리 알려, 딥러닝 학습의 기반을 마련했습니다.
    *   **볼츠만 머신(Boltzmann Machine) 및 딥 빌리프 네트워크(DBN) 개발:** 심층 신경망(Deep Neural Network)이 어떻게 효율적으로 학습할 수 있는지에 대한 초기 모델들을 제시하며 '딥러닝'이라는 분야의 문을 열었습니다.
    *   **AlexNet 개발 지도:** 2012년 이미지 인식 경진대회(ILSVRC)에서 압도적인 성능으로 우승한 'AlexNet'의 개발을 지도했습니다. 이 사건은 AI 분야에 딥러닝의 가능성을 증명하며 'AI의 빅뱅'을 일으킨 계기가 되었습니다.

---

### 2. 얀 르쿤 (Yann LeCun)

![얀 르쿤](https://upload.wikimedia.org/wikipedia/commons

In [12]:
# 토큰 사용량 확인하기
print("=== 응답 ===")
print(response.text)
print("\n=== 토큰 사용량 ===")
print(f"입력 토큰: {response.usage_metadata.prompt_token_count}")
print(f"출력 토큰: {response.usage_metadata.candidates_token_count}")
print(f"생각 토큰: {response.usage_metadata.thoughts_token_count}")
print(f"총 토큰: {response.usage_metadata.total_token_count}")

=== 응답 ===
## AI 분야의 3대 거장과 주요 업적

AI, 특히 딥러닝 분야의 발전을 이끈 "딥러닝의 아버지" 또는 "AI의 대부들(Godfathers of AI)"로 불리는 세 명의 석학이 가장 상징적입니다. 이들은 2018년, 컴퓨터 과학의 노벨상이라 불리는 **튜링상(Turing Award)**을 공동 수상하며 그 공로를 세계적으로 인정받았습니다.

---

### 1. 제프리 힌튼 (Geoffrey Hinton)

![제프리 힌튼](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3b/Geoffrey_Hinton_at_the_Royal_Society.jpg/440px-Geoffrey_Hinton_at_the_Royal_Society.jpg)

**"딥러닝의 개념적 토대를 마련한 선구자"**

*   **소속:** 토론토 대학교 명예교수, 前 구글 부사장 및 엔지니어링 펠로우
*   **주요 업적:**
    *   **역전파 알고리즘(Backpropagation)의 대중화:** 신경망이 스스로 오류를 수정하며 학습할 수 있게 하는 핵심 알고리즘입니다. 힌튼 교수는 이 알고리즘의 중요성을 재발견하고 널리 알려, 딥러닝 학습의 기반을 마련했습니다.
    *   **볼츠만 머신(Boltzmann Machine) 및 딥 빌리프 네트워크(DBN) 개발:** 심층 신경망(Deep Neural Network)이 어떻게 효율적으로 학습할 수 있는지에 대한 초기 모델들을 제시하며 '딥러닝'이라는 분야의 문을 열었습니다.
    *   **AlexNet 개발 지도:** 2012년 이미지 인식 경진대회(ILSVRC)에서 압도적인 성능으로 우승한 'AlexNet'의 개발을 지도했습니다. 이 사건은 AI 분야에 딥러닝의 가능성을 증명하며 'AI의 빅뱅'을 일으킨 계기가 되었습니다.

---

### 2. 얀 르쿤 (Yann LeCun)

![얀 르쿤](https://upload.wikimedia.org/wikipe

## 3. 구글 검색으로 그라운딩

In [16]:
from google import genai
from google.genai import types

# Configure the client
client = genai.Client(api_key=GOOGLE_API_KEY)

# Define the grounding tool
# GoogleSearch 사용할 수 있도록 설정
grounding_tool = types.Tool(google_search=types.GoogleSearch())

# Configure generation settings
# temperature, thinkingbuget 말고도 tool을 연결가능
# tool을 사용하지 않으면 학습때 사용됐던 정보만 보이고 google search를 사용하면 최신 정보를 가져올 수 있음
config = types.GenerateContentConfig(tools=[grounding_tool])

# Make the request
response = client.models.generate_content(
    # 사용모델 설정
    model="gemini-2.5-flash",
    # 프롬프트 입력
    # contents="2025년에 했던 대한민국 선거 날짜는?",
    contents="오늘 날씨는?",
    # google search를 사용 설정
    config=config,
)

# Print the grounded response
print(response.text)

2025년 8월 11일 월요일, 오늘은 전국적으로 흐리고 비가 내리는 곳이 많겠습니다. 특히 충청 이남을 중심으로 비가 이어지겠으며, 남해안에는 내일까지 100mm 이상, 전남과 경남에는 최대 80mm의 많은 비가 예상됩니다. 제주 동부에는 호우주의보가 발령되었으며, 현재 제주 일부 지역과 경남 밀양에는 시간당 20mm 안팎의 강한 비가 쏟아지고 있습니다. 경기도와 강원도 일부 지역에는 5~40mm, 대구·경북 지역에는 최대 40mm, 인천·경기도 및 서해5도에는 20~60mm (일부 지역 80mm 이상)의 비가 예보되었습니다.

기온은 아침 최저 21~26도, 낮 최고 27~30도의 분포를 보이겠습니다. 서울은 29.5도를 기록하며 대부분 지역이 30도를 밑돌았지만, 당분간 한낮에 28~32도 안팎의 평년 수준 기온이 이어지겠습니다. 비가 내리는 지역은 습도가 높아 체감온도가 31도 이상으로 올라 무덥게 느껴지겠습니다. 경기도 파주에는 오후 5시를 기해 폭염주의보가 발효되었습니다.

미세먼지 농도는 전국적으로 '좋음'에서 '보통' 수준을 보이겠습니다. 또한, 바닷물의 높이가 높은 대조기이므로 만조 시 해안가 침수에 각별히 유의해야 합니다.


In [17]:
# Google Search를 사용하여 얻은 답변에 출처(citation)를 추가하는 함수
def add_citations(response):
    text = response.text
    supports = response.candidates[0].grounding_metadata.grounding_supports
    chunks = response.candidates[0].grounding_metadata.grounding_chunks

    # Sort supports by end_index in descending order to avoid shifting issues when inserting.
    sorted_supports = sorted(supports, key=lambda s: s.segment.end_index, reverse=True)

    for support in sorted_supports:
        end_index = support.segment.end_index
        if support.grounding_chunk_indices:
            # Create citation string like [1](link1)[2](link2)
            citation_links = []
            for i in support.grounding_chunk_indices:
                if i < len(chunks):
                    uri = chunks[i].web.uri
                    citation_links.append(f"[{i + 1}]({uri})")

            citation_string = ", ".join(citation_links)
            text = text[:end_index] + citation_string + text[end_index:]

    return text


# Assuming response with grounding metadata
text_with_citations = add_citations(response)
print(text_with_citations)

2025년 8월 11일 월요일, 오늘은 전국적으로 흐리고 비가 내리는 곳이 많겠습니다. 특히 충청 이남을 중심으로 비가 이어지겠으며, 남해안에는 내일까지 100mm 이상, 전남과 경남에는 최[1](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF30xOlzL_v16WYgNUrsgXSLmQXudQYI2TWPCsqMSvDD_H6eI56vD339s8L7YZPPnc5OWbWv_Z68RKw6HKsVcfYFWbQpl4Q1YHt1yXSafK_gIUP58LTvj4BnteIjRrmMy-JH_9bY1qrNw==), [2](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEhFKevRWGv4UyaBE-k6AVCAazWVMD8L7xLyOAZmVfFsNLjIAL8_yguehNo_GZEC52X5tNxYE7Tw-Ru727GvyLW8FuEEHPZtaYwEzVFtK-ptG_jykMRiC2b2Ik3lMHbGj2HMhdzhzLXz5vU-0nHbr-T9UXGTw7iIiBnKsfWT1oY), [3](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE_-xgYka__Dd1ddCf9WwpGBBDBz0MUNMt7aS6FZ35KjLFzf3SApORmUoLNcjCbRAapq1ijjHEzXxwIjhhIToFIN2_v7Qw82FF5XMl39kHEddqFVSyAnYC8C6aEFXjODlZlOCJPhkQ=)대 80mm의 많은 비가 예상됩니다. 제주 동부에는 호우주의보가 발령되었으며, 현재 제주 일부 지역과 경남 밀양에는 시간당 20mm 안팎의 강한 비가 쏟아지고 있습니다. 경기도와 강원도 일부 지역에는 5~40mm, 대구·경북 지역에는 최대 40mm, 인천·경기도 및 서해5도에는 20~60mm (일부 지역 80mm [4](https://vertexaisearch.cloud.google.co

## 4. 음성 생성

In [18]:
from google import genai
from google.genai import types
import wave

# Set up the wave file to save the output:
 # filename 생성될 파일명
 # wb :쓰기모드 바이너리, channels=1 mono
 #  rate=24000 8kHz보다 2배정도면 음질을 담아도 적당 여기서는 더 할 당함 24kHz
 # pcm 방식 녹음
 # ==> wave파일을 만드는 조건
def wave_file(filename, pcm, channels=1, rate=24000, sample_width=2):
   with wave.open(filename, "wb") as wf:
      wf.setnchannels(channels)
      wf.setsampwidth(sample_width)
      wf.setframerate(rate)
      wf.writeframes(pcm)

client = genai.Client(api_key=GOOGLE_API_KEY)

# 모델을 설정
response = client.models.generate_content(
   model="gemini-2.5-flash-preview-tts", # 음성을 만들어주는 전용모델
   #contents="Say cheerfully: Have a wonderful day!",
   contents="밝은 목소리로: 좋은하루 되세요!", # :콜론은 참조의 의미
   config=types.GenerateContentConfig(
      response_modalities=["AUDIO"],
      speech_config=types.SpeechConfig(
         voice_config=types.VoiceConfig(
            prebuilt_voice_config=types.PrebuiltVoiceConfig(
               voice_name='Achird', # voice name
            )
         )
      ),
   )
)

# data 생성
data = response.candidates[0].content.parts[0].inline_data.data

file_name='kor-good-morning.wav'
wave_file(file_name, data) # Saves the file to current directory

## 5. 영상 생성

In [19]:
import time
from google import genai
from google.genai import types

client = genai.Client(api_key=GOOGLE_API_KEY)

# 프롬프트로 입력할 문자열
# prompt = """A close up of two people staring at a cryptic drawing on a wall, torchlight flickering.
# A man murmurs, 'This must be it. That's the secret code.' The woman looks at him and whispering excitedly, 'What did you find?'"""
prompt = """ 횃불이 깜빡이는 가운데, 두 사람이 벽에 그려진 의미심장한 그림을 클로즈업해서 쳐다보고 있다.
 한 남자가 중얼거린다. "이게 틀림없어. 저게 비밀 암호야." 여자가 흥분한 목소리로 속삭이며 그를 쳐다본다. "어떻게 찾았은거야?"
 남자가 다시 대답한다. "나는 다 알 수 있어. 이건 개꿈이거든"
"""

try:
    # 비디오 생성 요청
    operation = client.models.generate_videos(
        model="veo-3.0-generate-preview", # VEO3 모델 지정
        prompt=prompt, # 프롬프트 입력
    )

    # Poll the operation status until the video is ready.
    max_attempts = 60  # 최대 10분 대기 (10초씩 60번)
    attempts = 0

    # 작업 완료 대기
    while not operation.done and attempts < max_attempts:
        print(
            f"Waiting for video generation to complete... (attempt {attempts + 1}/{max_attempts})"
        )
        time.sleep(10)
        # 작업의 현재 상태를 가져옴
        operation = client.operations.get(operation)
        attempts += 1
    # 비디오 생성 과정에서 오류가 발생하면 해당 오류 메시지를 출력
    if not operation.done:
        print("Video generation timed out after 10 minutes")
    elif operation.response is None:
        print("Video generation completed but no response received")
        print(f"Operation status: {operation}")
    elif (
        not hasattr(operation.response, "generated_videos")
        or not operation.response.generated_videos
    ):
        print("Video generation completed but no videos were generated")
        print(f"Response: {operation.response}")
    else:
        # Download the generated video.
        generated_video = operation.response.generated_videos[0]
        client.files.download(file=generated_video.video)
        generated_video.video.save("dialogue_example.mp4")
        print("Generated video saved to dialogue_example.mp4")

except Exception as e:
    print(f"Error occurred during video generation: {e}")
    print(f"Error type: {type(e).__name__}")

Waiting for video generation to complete... (attempt 1/60)
Waiting for video generation to complete... (attempt 2/60)
Waiting for video generation to complete... (attempt 3/60)
Waiting for video generation to complete... (attempt 4/60)
Waiting for video generation to complete... (attempt 5/60)
Waiting for video generation to complete... (attempt 6/60)
Waiting for video generation to complete... (attempt 7/60)
Generated video saved to dialogue_example.mp4
